In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#https://www.kaggle.com/competitions/premiumpulse-risk-modeling/overview

# === MANIPULACIÓN DE DATOS ===
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# === VISUALIZACIÓN ===
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# === PREPROCESAMIENTO ===
from sklearn.preprocessing import (
    StandardScaler, MinMaxScaler, RobustScaler,
    LabelEncoder, OneHotEncoder, OrdinalEncoder,
    PowerTransformer, QuantileTransformer
)
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# === DIVISIÓN DE DATOS ===
from sklearn.model_selection import (
    train_test_split, cross_val_score, cross_validate,
    StratifiedKFold, KFold, GridSearchCV, RandomizedSearchCV,
    validation_curve, learning_curve
)

# === MODELOS DE CLASIFICACIÓN ===
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import (
    RandomForestClassifier, ExtraTreesClassifier,
    GradientBoostingClassifier, AdaBoostClassifier,
    VotingClassifier, BaggingClassifier
)
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

# === MODELOS DE REGRESIÓN ===
from sklearn.linear_model import (
    LinearRegression, Ridge, Lasso, ElasticNet,
    SGDRegressor, BayesianRidge, HuberRegressor
)
from sklearn.ensemble import (
    RandomForestRegressor, ExtraTreesRegressor,
    GradientBoostingRegressor, AdaBoostRegressor,
    VotingRegressor, BaggingRegressor
)
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

# === CLUSTERING ===
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture

# === REDUCCIÓN DE DIMENSIONALIDAD ===
from sklearn.decomposition import PCA, TruncatedSVD, FastICA
from sklearn.manifold import TSNE
from sklearn.feature_selection import (
    SelectKBest, SelectFromModel, RFE, RFECV,
    chi2, f_classif, f_regression, mutual_info_classif
)

# === MÉTRICAS ===
from sklearn.metrics import (
    # Clasificación
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, roc_auc_score,
    roc_curve, precision_recall_curve, auc, log_loss,
 
    # Regresión
    mean_squared_error, mean_absolute_error, r2_score,
    mean_squared_log_error, mean_absolute_percentage_error,
    
    # Clustering
    silhouette_score, adjusted_rand_score, calinski_harabasz_score
)

# === GRADIENT BOOSTING 

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier, CatBoostRegressor

# === ANÁLISIS ESTADÍSTICO ===

import scipy.stats as stats
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# === TIME SERIES ===

from prophet import Prophet

# === OPTIMIZACIÓN DE HIPERPARÁMETROS ===

import optuna

# === UTILIDADES ===
import os
import sys
import json
import pickle
import joblib
from datetime import datetime, timedelta
import itertools
from collections import Counter
import gc

# === CONFIGURACIÓN DE PANDAS ===
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)
pd.set_option('display.width', None)

# === CONFIGURACIÓN DE MATPLOTLIB ===
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# === CONFIGURACIÓN DE SEABORN ===
sns.set_palette("husl")
sns.set_style("whitegrid")

# === CONFIGURACIÓN DE NUMPY ===
np.random.seed(1722)

In [ ]:
train = pd.read_csv('/kaggle/input/premiumpulse-risk-modeling/train.csv')
test = pd.read_csv('/kaggle/input/premiumpulse-risk-modeling/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
#lo primero sera ver como correr rapido un RandomForest y ver como esta la cosa
#variable a predecir "Premium Amount"
#ver si sacar "id" y la agrego al final
#Pasar a logaritmo el "Annual Income" y tal vez el "Credit Score"
#Ver los NaN cuantos son y como proceder
#Variables categoricas:"Marital Status","Location","Property Type"
#Variables categoricas ordinales: "Education Level","Occupation","Policy Type","Customer Feedback","Exercise Frequency"
#Analisis de correlaciones
#separar Policy Start Date en varias columnas año/mes/dia/hora ver si es relevante el dia como lun/mar/mier o por semanas
#ver si las variables numericas parecieran seguir algun tipo de distribucion normal/poisson/uniforme etc..
# Variable "Gender" por un tema de discriminacion la saco o no ? , espero a ver la importancia en un random forest para decidir

In [ ]:
train.describe(include="all")

In [ ]:
test.describe(include="all")# los valores entre test y train parecieran salir de una misma poblacion por como estan distribuidos , parece sintetico/ficticio

In [ ]:
#train.isnull().sum()
#train["Age"].value_counts() parece seguir una distribucion uniforme entre 18 a 64 años ,
# voy a reemplazar los NaN en la edad con la mediana , pero voy a crear una columna binaria para que el modelo sepa si habia NaN en la Edad
# es por si en esas edad en particular el cambio que hice perjudica al modelo , ya sea porque asigne mas riesgo a esa edad o saque conclusiones
# que no son correctas debido a mi manipulacion y el hecho de que hace ruido que la edad este distribuida uniformemente entre todos los rangos

train1 = train.copy()
test1 = test.copy()

train1["Age_NaN"] = train["Age"].isna().astype(int)
test1["Age_NaN"] = test["Age"].isna().astype(int)


train1["Age"] = train["Age"].fillna(train["Age"].median())

#train["Age"].value_counts()

test1["Age"] = test["Age"].fillna(test["Age"].median())

#test1["Age"].value_counts()

train1["Age_NaN"] = train["Age"].isna().astype(int)
test1["Age_NaN"] = test["Age"].isna().astype(int)

#train1.head(10)
#test1.head(10)

In [ ]:
#train1.head(10)
train1["Age_NaN"].value_counts()
#test1["Age_NaN"].value_counts()

In [ ]:
train1["An_Income_NaN"] = train["Annual Income"].isna().astype(int)
test1["An_Income_NaN"] = test["Annual Income"].isna().astype(int)


train1["Annual Income"] = train["Annual Income"].fillna(train["Annual Income"].median())
test1["Annual Income"] = test["Annual Income"].fillna(test["Annual Income"].median())


train1["Log Annual Income"] = np.log(train1["Annual Income"])
test1["Log Annual Income"] = np.log(test1["Annual Income"])

train1 = train1.drop("Annual Income", axis = 1)
test1 = test1.drop("Annual Income", axis = 1)

In [ ]:
#train1.head(10)
train1["An_Income_NaN"].value_counts()
test1["An_Income_NaN"].value_counts()

In [ ]:
train1.head()

In [ ]:
#para el "Credit Score" le hago categorias , uso Ordinal Scoring

def categorize_Credit_Score(score):
    if pd.isna(score):
        return 'X'  # Sin historial
    elif score >= 800:
        return 'A'  # Excepcional (800-850)
    elif score >= 750:
        return 'B'  # Muy Bueno
    elif score >= 700:
        return 'C'  # Bueno
    elif score >= 650:
        return 'D'  # Regular+
    elif score >= 600:
        return 'E'  # Regular
    elif score >= 550:
        return 'F'  # Malo+
    elif score >= 500:
        return 'G'  # Malo
    elif score >= 400:
        return 'H'  # Muy Malo
    else:  # 300-399
        return 'I'  # Pésimo

train1['Credit_Grade'] = train1['Credit Score'].apply(categorize_Credit_Score)
test1['Credit_Grade'] = test1['Credit Score'].apply(categorize_Credit_Score)


grade_map = {'A':9, 'B':8, 'C':7, 'D':6, 'E':5, 'F':4, 'G':3, 'H':2, 'I':1, 'X':0}
train1['Credit Score'] = train1['Credit_Grade'].map(grade_map)
test1['Credit Score'] = test1['Credit_Grade'].map(grade_map)

In [ ]:
train1.head()

In [ ]:
train1 = train1.drop("Credit_Grade", axis = 1)
test1 = test1.drop("Credit_Grade", axis = 1)

test1.head()

In [ ]:
train2 = train1.copy()
test2 = test1.copy()

In [ ]:
#Reemplazo NaN por Unknown

train2['Marital Status'] = train2['Marital Status'].fillna('Unknown')
test2['Marital Status'] = test2['Marital Status'].fillna('Unknown')

train2.describe(include="all")

In [ ]:
plt.figure(figsize=(7,2)) 
plt.title("titulo")
sns.boxplot(x=train2["Number of Dependents"])
print(train2["Number of Dependents"].describe())

In [ ]:
plt.figure(figsize=(15,3)) 
plt.title("Number of Dependents")
sns.histplot(train2["Number of Dependents"]) 

In [ ]:
#para 'Number of Dependents' voy a poner -1 en los NaN y crear una columna que lo aclare

train2['Number of Dependents'] = train2['Number of Dependents'].fillna(-1)
test2['Number of Dependents'] = test2['Number of Dependents'].fillna(-1)


train2['Dependents_NaN'] = (train2['Number of Dependents'] == -1).astype(int)
test2['Dependents_NaN'] = (test2['Number of Dependents'] == -1).astype(int)


In [ ]:
#test2.head()
train2['Dependents_NaN'].value_counts()

In [ ]:
train2.describe(include="all")

In [ ]:
# para Occupation reemplazo NaN por Unknown

train2['Occupation'] = train2['Occupation'].fillna('Unknown')
test2['Occupation'] = test2['Occupation'].fillna('Unknown')

test2.describe(include="all")

In [ ]:
train2.describe(include="all")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Boxplot
ax1.boxplot(train2['Health Score'].dropna())
ax1.set_title('Health Score - Boxplot')

# Histograma
ax2.hist(train2['Health Score'].dropna(), bins=20, edgecolor='black')
ax2.set_title('Health Score - Histogram')

plt.tight_layout()
plt.show()

print(train2['Health Score'].describe())
print(f"NaN: {train2['Health Score'].isna().sum()}")

In [ ]:
#Crear rangos y columna nueva para los NaN

# Binning
train2['Health_Score_Binned'] = pd.cut(train2['Health Score'], bins=8, labels=range(8)).astype(float)
train2['Health_Score_Binned'] = train2['Health_Score_Binned'].fillna(-1)
test2['Health_Score_Binned'] = pd.cut(test2['Health Score'], bins=8, labels=range(8)).astype(float)
test2['Health_Score_Binned'] = test2['Health_Score_Binned'].fillna(-1)


# Indicator
train2['Health_NaN'] = (train2['Health_Score_Binned'] == -1).astype(int)
test2['Health_NaN'] = (test2['Health_Score_Binned'] == -1).astype(int)

train2 = train2.drop("Health Score", axis = 1)
test2 = test2.drop("Health Score", axis = 1)


train2.head()

In [ ]:

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Boxplot
ax1.boxplot(train2['Previous Claims'].dropna())
ax1.set_title('Previous Claims - Boxplot')

# Histograma
ax2.hist(train2['Previous Claims'].dropna(), bins=2, edgecolor='black')
ax2.set_title('Previous Claims - Histogram')

plt.tight_layout()
plt.show()

print(train2['Previous Claims'].describe())
print(f"NaN: {train2['Previous Claims'].isna().sum()}")



In [ ]:
#Previous Claim parece una poisson , aplico -1 en los NaN que son bastantes y agrego Columna NaN binaria

train2['Previous Claims'] = train2['Previous Claims'].fillna(-1)
test2['Previous Claims'] = test2['Previous Claims'].fillna(-1)


train2['Previous_Claims_NaN'] = (train2['Previous Claims'] == -1).astype(int)
test2['Previous_Claims_NaN'] = (test2['Previous Claims'] == -1).astype(int)

train2.head()

In [ ]:
train3 = train2.copy()
test3 = test2.copy()

train3.describe(include = "all")

In [ ]:

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Boxplot
ax1.boxplot(train3['Vehicle Age'].dropna())
ax1.set_title('Vehicle Age - Boxplot')

# Histograma
ax2.hist(train3['Vehicle Age'].dropna(), bins=2, edgecolor='black')
ax2.set_title('Vehicle Age - Histogram')

plt.tight_layout()
plt.show()

print(train3['Vehicle Age'].describe())
print(f"NaN: {train3['Vehicle Age'].isna().sum()}")

In [ ]:
# por solo 6 valores faltantes podria hacer lo que quiera , ya sea eliminar las filas , reemplazar por la media o mediana , etc
# elijo reemplazar por la mediana para conservar 1,2kk de datos

train3['Vehicle Age'] = train3['Vehicle Age'].fillna(train3["Vehicle Age"].median())
test3['Vehicle Age'] = test3['Vehicle Age'].fillna(test3["Vehicle Age"].median())







In [ ]:
train3.describe(include="all")

In [ ]:
#insurance 1 solo valor , al ser alguien de 64 años osea la edad maxima , le pongo 1 de duration , porque en el dataset no se ve a nadie 
#con una edad superior con poliza activa

nan_row = train3[train3['Insurance Duration'].isna()]
print(nan_row)

train3['Insurance Duration'] = train3['Insurance Duration'].fillna(1)


In [ ]:
# son 2 casos distintos , pero 2 datos entre 800k no me va a cambiar ,asi que tambien pongo 1 

nan_row = test3[test3['Insurance Duration'].isna()]
print(nan_row)

test3['Insurance Duration'] = test3['Insurance Duration'].fillna(1)

In [ ]:
# para Policy Start date extraigo  AÑO/TRIMESTRE/MES/DIA/ DIA SEMANA  hora/minutos/segundos puede ser ruido , capaz las horas por querer ver
# el horario laboral de cuando se hacen mas inicio de poliza
# luego elimino la fecha

# Convertir a datetime si no lo está
train3['Policy Start Date'] = pd.to_datetime(train3['Policy Start Date'])
test3['Policy Start Date'] = pd.to_datetime(test3['Policy Start Date'])


train3['Year'] = train3['Policy Start Date'].dt.year
train3['Quarter'] = train3['Policy Start Date'].dt.quarter
train3['Month'] = train3['Policy Start Date'].dt.month
train3['Day'] = train3['Policy Start Date'].dt.day
train3['DayOfWeek'] = train3['Policy Start Date'].dt.dayofweek  # 0=Lunes, 6=Domingo
train3['Hour'] = train3['Policy Start Date'].dt.hour

test3['Year'] = test3['Policy Start Date'].dt.year
test3['Quarter'] = test3['Policy Start Date'].dt.quarter
test3['Month'] = test3['Policy Start Date'].dt.month
test3['Day'] = test3['Policy Start Date'].dt.day
test3['DayOfWeek'] = test3['Policy Start Date'].dt.dayofweek
test3['Hour'] = test3['Policy Start Date'].dt.hour

In [ ]:
train3 = train3.drop('Policy Start Date', axis=1)
test3 = test3.drop('Policy Start Date', axis=1)


train3.head()

In [ ]:
train3.describe(include="all")

In [ ]:
# para customer feedback , reemplazo con unknowk los NaN

train3['Customer Feedback'] = train3['Customer Feedback'].fillna('Unknown')
test3['Customer Feedback'] = test3['Customer Feedback'].fillna('Unknown')




In [ ]:
train3.describe(include="all")

In [ ]:
def quick_info(df):
    """Información rápida del dataset"""
    print(f"📊 Dataset Shape: {df.shape}")
    print(f"🔢 Columnas numéricas: {df.select_dtypes(include=[np.number]).columns.tolist()}")
    print(f"📝 Columnas categóricas: {df.select_dtypes(include=['object']).columns.tolist()}")
    print(f"❌ Valores nulos por columna:")
    print(df.isnull().sum()[df.isnull().sum() > 0])


quick_info(train3)

In [ ]:
quick_info(test3)

In [ ]:
# ahora me tengo que encargar de hacer one-hot encoding y/o label encoding

# Ordinales - Label Encoding
from sklearn.preprocessing import LabelEncoder

ordinal_cols = ['Education Level', 'Customer Feedback', 'Exercise Frequency','Policy Type']
for col in ordinal_cols:
    le = LabelEncoder()
    train3[col] = le.fit_transform(train3[col].astype(str))
    test3[col] = le.transform(test3[col].astype(str))

# Nominales - One-Hot Encoding  
nominal_cols = ['Gender', 'Marital Status', 'Occupation', 'Location', 'Smoking Status', 'Property Type']
train_encoded = pd.get_dummies(train3, columns=nominal_cols, drop_first=True)
test_encoded = pd.get_dummies(test3, columns=nominal_cols, drop_first=True)

In [ ]:
quick_info(train_encoded)

In [ ]:
quick_info(test_encoded)

In [ ]:
#saco "id" que no lo utilizo ahora , solo lo uso  en la submission
train_ids = train_encoded['id'].copy()
test_ids = test_encoded['id'].copy()

# Al final, para submission:
#submission = pd.DataFrame({
#    'id': test_ids,
#    'prediction': predictions
#})


y = train_encoded["Premium Amount"]
X = train_encoded.drop(["Premium Amount","id"], axis=1)

X_t = test_encoded.drop('id', axis=1)

# Split para validación final
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1722)

# Grid de parámetros con estos parametros tardo mucho y tiro error
#param_grid = {
#    'n_estimators': [100, 200, 300],
#    'max_depth': [10, 15, 20, None],
#    'min_samples_split': [2, 5, 10],
#    'min_samples_leaf': [1, 2, 4],
#    'max_features': ['sqrt', 'log2', None]
#}

#param_grid = {
#    'n_estimators': [100, 200],
#    'max_depth': [10, 20],
#    'min_samples_split': [2, 10],
#    'min_samples_leaf': [1, 4],
#    'max_features': ['sqrt', None]
#}
#random_search = RandomizedSearchCV(
#    RandomForestRegressor(random_state=1722, n_jobs=-4),
#    param_grid,
#    n_iter=20,
#    cv=3,
#    scoring='neg_mean_squared_log_error', 
#    n_jobs=-4,
#    verbose=3,
#    random_state=1722
#)


In [ ]:
#%%time
#random_search.fit(X_train, y_train)


In [ ]:
# Mejores parámetros
#print("Mejores parámetros:", random_search.best_params_)
#print("Mejor score CV:", -random_search.best_score_)  # Negativo por el 'neg_'

# Evaluar en validación con RMSLE
#best_rf = random_search.best_estimator_
#y_pred = best_rf.predict(X_val)
#rmsle = np.sqrt(mean_squared_log_error(y_val, y_pred))
#print(f"RMSLE: {rmsle:.4f}")

#Mejores parámetros: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': None, 'max_depth': 10}
#Mejor score CV: 1.28097918942279
#RMSLE: 1.1405 con este score entraria en el puesto 13 , dado que la competencia se reinicia cada cierto tiempo 

In [ ]:
#best_model =RandomForestRegressor(
#    random_state=1722,
#    n_jobs=-1,
#    n_estimators=100,
#    max_depth=10,
#    min_samples_split=10,
#    min_samples_leaf=2,
#    max_features="sqrt"
#)

#best_model.fit(X, y)

In [ ]:
#predictions = best_model.predict(X_t)

In [ ]:
#submission = pd.DataFrame({
#    'id': test_ids,  # Los IDs que guardaste
#    'Premium Amount': predictions  # Cambia por el nombre correcto de la columna target
#})

#submission.to_csv('submission.csv', index=False)
#print("Submission creado!")
#print(submission.head())

In [ ]:
#submission.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
#dejo por aca el notebook , puedo probar XGBoost/LGBM/Catboot y DNN / ensemble y/o hacer un feature eng , por ser una competencia con pocos participantes
# no se puede comparar y ver en que fueron mejor los demas competidores y obtener ideas
# prefiero continuar con otros para seguir aprendiendo

In [ ]:
#v5
#param_grid = {
#    'n_estimators': [50, 90, 140],
#    'max_depth': [7, 14],
#    'min_samples_split': [2, 4],
#    'min_samples_leaf': [1, 4],
#    'max_features': ['sqrt', 'log2']
#}
#random_search1 = RandomizedSearchCV(
#    RandomForestRegressor(random_state=1, n_jobs=-1),
#    param_grid,
#    n_iter=10,
#    cv=3,
#    scoring='neg_mean_squared_log_error', 
#    n_jobs=-1,
#    verbose=3,
#    random_state=1
#)

In [ ]:
#%%time
#random_search1.fit(X, y)

In [ ]:
#random_best_params1 = random_search1.best_params_

#print("Mejores parámetros:", random_search1.best_params_)
#print("Mejor score CV:", -random_search1.best_score_)  

In [ ]:
#random1 = RandomForestRegressor(
#    random_state=1,
#    n_jobs=-1,
#    n_estimators=random_best_params1['n_estimators'],
#    max_depth=random_best_params1['max_depth'],
#    min_samples_split=random_best_params1['min_samples_split'],
#    min_samples_leaf=random_best_params1['min_samples_leaf'],
#    max_features=random_best_params1['max_features']
#)


In [ ]:
#%%time
#random1.fit(X, y)

In [ ]:
#predictions1 = random1.predict(X_t)

In [ ]:
#submission = pd.DataFrame({
#    'id': test_ids,  # Los IDs que guardaste
#    'Premium Amount': predictions1  # Cambia por el nombre correcto de la columna target
#})

#submission.to_csv('submission.csv', index=False)
#print("Submission creado!")
#print(submission.head())

In [ ]:
#submission.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
#v6
#param_grid = {
#    'n_estimators': [50, 90, 140],
#    'max_depth': [7, 14],
#    'min_samples_split': [2, 4],
#    'min_samples_leaf': [1, 4],
#    'max_features': ['sqrt', 'log2']
#}
#random_search2 = RandomizedSearchCV(
#    RandomForestRegressor(random_state=2, n_jobs=-1),
#    param_grid,
#    n_iter=10,
#    cv=3,
#    scoring='neg_mean_squared_log_error', 
#    n_jobs=-1,
#    verbose=3,
#    random_state=2
#)

In [ ]:
#%%time
#random_search2.fit(X, y)

In [ ]:
#random_best_params2 = random_search2.best_params_

#print("Mejores parámetros:", random_search2.best_params_)
#print("Mejor score CV:", -random_search2.best_score_)  

In [ ]:
#random2 = RandomForestRegressor(
#    random_state=2,
#    n_jobs=-1,
#    n_estimators=random_best_params2['n_estimators'],
#    max_depth=random_best_params2['max_depth'],
#    min_samples_split=random_best_params2['min_samples_split'],
#    min_samples_leaf=random_best_params2['min_samples_leaf'],
#    max_features=random_best_params2['max_features']
#)

In [ ]:
#%%time
#random2.fit(X, y)

In [ ]:
#predictions2 = random2.predict(X_t)

In [ ]:
#submission = pd.DataFrame({
#    'id': test_ids,  # Los IDs que guardaste
#    'Premium Amount': predictions2  # Cambia por el nombre correcto de la columna target
#})

#submission.to_csv('submission.csv', index=False)
#print("Submission creado!")
#print(submission.head())

In [ ]:
#submission.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
#v7 xgboost hecho en otro lado
#Mejores parámetros: {'subsample': 0.9, 'reg_lambda': 2, 'reg_alpha': 1, 'n_estimators': 100, 'max_depth': 9, 'learning_rate': 0.1, 'colsample_bytree': 1.0}
#Mejor score: 1.288746636454776
# va con ensemble , pero aunque mejore un poco , si o si feature eng para mejorar mas



In [ ]:
Xgb1 = xgb.XGBRegressor(
    random_state=2,
    subsample  = 0.9,
    reg_lambda = 2,
    reg_alpha = 1,
    n_estimators = 100,
    max_depth = 9,
    learning_rate = 0.1,
    colsample_bytree = 1.0
)

Xgb1.fit(X, y)

In [ ]:
random1 = RandomForestRegressor(
    random_state=1,
    n_jobs=-1,
    n_estimators= 100,
    max_depth= 10,
    min_samples_split= 2,
    min_samples_leaf= 4,
    max_features= None
)

random1.fit(X, y)

In [ ]:

# Hacer predicciones con ambos
pred_rf = random1.predict(X_t)
pred_xgb = Xgb1.predict(X_t)

# Ensemble simple: promedio
predictions = (pred_rf + pred_xgb) / 2

# Crear submission
submission = pd.DataFrame({
    'id': test_ids,
    'Premium Amount': predictions
})

submission.to_csv('submission.csv', index=False)
print("Submission creado!")
print(submission.head())

In [ ]:
submission.to_csv('/kaggle/working/submission.csv', index=False)
# 1.13569 RMSLE logro en el score de la competencia , los anteriores era 1.1519
# si o si para mejorar es feature eng / feature importante y trabajar sobre esas columnas y ordenarlas y mas